In [1]:
from sklearn.externals import joblib
import nltk
import re
from konlpy.utils import pprint
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from konlpy.tag import Komoran
from konlpy.tag import Hannanum

In [2]:
clf = joblib.load('classify.model') 
cate_dict = joblib.load('cate_dict.dat')
vectorizer = joblib.load('vectorizer.dat')

In [3]:
kkma = Kkma()
twitter = Twitter()
hannanum = Hannanum()
KOREAN_INDEX = 1
KOREAN_NAME = 0

In [4]:
class CUtility :
    @staticmethod
    def reSubString(text, eraseString, regularExpression) :
        pattern = re.compile(regularExpression, re.U)
        return pattern.sub(eraseString, text)
    
    @staticmethod
    def reGroup(text, regularExpression) :
        m = re.search(regularExpression, text)
        print m.group()
        
    @staticmethod
    def reExploreList(listData, regularExpression) :
        tokens = re.compile(regularExpression)
        i = 0
        while(1) :
            if(i >= len(listData)) :
                break
            if tokens.search(listData[i]) is None :
                listData.pop(i)
                i = 0
            else : 
                i += 1
        return listData
    
    @staticmethod
    def fromStringToList(stringData) :
        stringList = stringData.split()
        i = 0
        while(1) :
            if(i >= len(stringList)) :
                break
            try :
                while(1) :
                    data = stringList[i].index(u",")
                    stringList[i] = stringList[i].replace(u",", "")
            except ValueError :
                a = 10
            i += 1
        return stringList
    @staticmethod
    def render(text) :
        pprint(text)

In [5]:
class CPreJob :
    # '%' 문자 제거
    def erasePercentage(self, text) :
        if '%' in text :
            percentageIndex = 0
            minIndex = 0
            try :
                while(1) :
                    percentageIndex = text.index('%', 0)
                    for minIndex in range(percentageIndex - 1, percentageIndex - 4, -1) :
                        if(text[minIndex].isdigit()) :
                            continue
                        else :
                            break
                    minIndex += 1
                    return text[0:minIndex] + text[percentageIndex + 1:len(text)]
            except ValueError :
                a = 10           
        return text
    
    # 괄호들 제거
    def eraseBracket(self, text) :
        closeDontNeedWordAsciiWord = [']', '[', ')', '(', '{', '}']
        for word in closeDontNeedWordAsciiWord :
            text = text.replace(word, " ")
        return text
         
    # 특징있는 문자들 찾기    
    def findFeaturePoint(self, text) :
        featurePoint = list()     
        bracketStart = ['(', '[', '{']
        bracketFinish = [')', ']', '}'] 
        for i in range(len(bracketStart)) :
            startIndex = 0
            while (1) :
                if startIndex >= len(text) :
                    break
                openIndex = text.find(bracketStart[i], startIndex)
                closeIndex = 0
                if (openIndex == -1) :
                    break
                else :
                    closeIndex = text.find(bracketFinish[i], startIndex)
                if (openIndex != -1 
                   and closeIndex != -1) :
                    featurePoint.append(text[openIndex:closeIndex + 1])
                    startIndex = closeIndex + 1 
                else :
                    break
        return featurePoint
        
    def eraseTrash(self, text) :
        return CUtility.reSubString(text, "", u'[^\uAC00-\uD7A3 a-zA-Z0-9./-]')
    
    def progress(self, text) :
        text = self.erasePercentage(text)
        text = self.eraseBracket(text)
        text = self.eraseTrash(text)
        return text

In [6]:
class CMorphEnglish :
    def cutTokenOne(self, tokens) :
        i = 0
        while(1) :
            if(i >= len(tokens)) :
                break

            if(len(tokens[i]) == 1) :
                tokens.pop(i)
                i = 0
            else :
                i += 1
        return tokens
    
    def NLTKTokenize(self, text) :
        englishTokens = nltk.word_tokenize(text)
        englishTagged = nltk.pos_tag(englishTokens)
        englishNouns = list()
        for tagIndex in range(len(englishTagged)) :
            if (englishTagged[tagIndex][0] == 'mm' or
            englishTagged[tagIndex][0] == 'M' or
            englishTagged[tagIndex][0] == 'MB' or
            englishTagged[tagIndex][0] == 'KB' or
            englishTagged[tagIndex][0] == 'G' or
            englishTagged[tagIndex][0] == 'GB' or
            englishTagged[tagIndex][0] == 'GHz' or
            englishTagged[tagIndex][0] == 'GT/s' or
            englishTagged[tagIndex][0] == 'TB') :
                englishNouns.append(englishTagged[tagIndex - 1][0] + englishTagged[tagIndex][0])
        englishNouns = CUtility.reExploreList(englishNouns, "[a-zA-Z]")
        return englishNouns
    
    def twitterTokenize(self, text) :
        stringList = list()
        twitterPos = twitter.pos(text)
        for posIndex in range(len(twitterPos)) :
            if posIndex == 0 :
                continue
            if posIndex + 1 >= len(twitterPos) :
                break
            if(twitterPos[posIndex][0] == '-') :
                stringList.append(twitterPos[posIndex - 1][0] + twitterPos[posIndex][0] + twitterPos[posIndex + 1][0])
        return stringList

    def wordTokenize(self, text) :    
        englishTokens = nltk.word_tokenize(text)
        englishTokens = CUtility.reExploreList(englishTokens, "[a-zA-Z]")
        englishTokens = self.cutTokenOne(englishTokens)
        return englishTokens
    
    def progress(self, text) :
        return self.wordTokenize(text)

In [7]:
class CMorphKorean :

    def KkmaTokens(self, text) :
        kkmaNounsList = kkma.nouns(text)
        kkmaPosList = kkma.pos(text)    
        for posIndex in range(len(kkmaPosList)) :
            if(kkmaPosList[posIndex][KOREAN_INDEX] == "NNP") :
                continue
            else :
                for nounsIndex in range(len(kkmaNounsList)) :
                    if(kkmaPosList[posIndex][KOREAN_NAME] ==
                          kkmaNounsList[nounsIndex]) :
                        kkmaNounsList.pop(nounsIndex)
                        break
        return kkmaNounsList
    
    def twitterTokens(self, text) :
        twitterNounsList = twitter.nouns(text)
        twitterPosList = twitter.pos(text)
        for posIndex in range(len(twitterPosList)) :
            if(twitterPosList[posIndex][KOREAN_INDEX] == "Noun") :
                if(len(twitterPosList[posIndex][KOREAN_NAME]) == 1) :
                    twitterNounsList.append(twitterPosList[posIndex - 1][KOREAN_NAME] + twitterPosList[posIndex][KOREAN_NAME])  
                else : 
                    continue     
            elif (twitterPosList[posIndex][KOREAN_INDEX] == "Adjective"
                    or twitterPosList[posIndex][KOREAN_INDEX] == "Verb"
                    or twitterPosList[posIndex][KOREAN_INDEX] == "Eomi") :
                twitterNounsList.append(twitterPosList[posIndex][KOREAN_NAME])
            elif (twitterPosList[posIndex][KOREAN_INDEX] == "Suffix" 
                 or twitterPosList[posIndex][KOREAN_INDEX] == "Josa") :
                twitterNounsList.append(twitterPosList[posIndex - 1][KOREAN_NAME] + twitterPosList[posIndex][KOREAN_NAME])   
            else :                
                for nounsIndex in range(len(twitterNounsList)) :
                    if(twitterPosList[posIndex][KOREAN_NAME] ==
                          twitterNounsList[nounsIndex]) :                            
                        twitterNounsList.pop(nounsIndex)
                        break
        nounsIndex = 0
        while(1) :
            if(nounsIndex >= len(twitterNounsList)) :
                break
            if(len(twitterNounsList[nounsIndex]) == 1) :
                if(twitterNounsList[nounsIndex] == u"기" or
                   twitterNounsList[nounsIndex] == u"원" or
                  twitterNounsList[nounsIndex] == u"형") :
                    twitterNounsList.pop(nounsIndex)
                    nounsIndex = 0           
                else :
                    nounsIndex += 1
            else :
                nounsIndex += 1
        return twitterNounsList
    
    def progress(self, text) :
        return self.twitterTokens(text)

In [8]:
class CMorphJob :
    englishJob = CMorphEnglish()
    koreanJob = CMorphKorean()
    
    def progress(self, text) :
        result = self.englishJob.progress(text) + self.koreanJob.progress(text)
        return result

In [9]:
class CJob :
    preProcessing = CPreJob()
    morphJob = CMorphJob()
    
    def preProgress(self, text) :
        return self.preProcessing.progress(text)
        
    def morphProgress(self, text) :
        return self.morphJob.progress(text) 
        
    def progress(self, text) :
        pointWord = self.preProcessing.findFeaturePoint(text)
        text = self.preProgress(text)
        text = self.morphProgress(text)
        return text

In [10]:
job = CJob()

In [11]:
joblib.dump(clf,'n_classify.model')

['n_classify.model',
 'n_classify.model_01.npy',
 'n_classify.model_02.npy',
 'n_classify.model_03.npy']

In [12]:
joblib.dump(cate_dict,'n_cate_dict.dat')
joblib.dump(vectorizer,'n_vectorizer.dat')

['n_vectorizer.dat']

In [13]:
cate_id_name_dict = dict(map(lambda (k,v):(v,k),cate_dict.items()))

In [ ]:
pred = clf.predict(vectorizer.transform(['[신한카드5%할인][서우한복] 아동한복 여자아동 금나래 (분홍)']))[0]
print cate_id_name_dict[pred]

패션의류;아동의류;한복


In [ ]:
from bottle import route, run, template,request,get, post
import  time
from threading import  Condition
_CONDITION = Condition()

@route('/classify')
def classify():
    print "classify called"
    img = request.GET.get('img','')
    name = request.GET.get('name', '').decode('utf-8')   
    name = job.progress(name)
    name = " ".join(name)
    pred = clf.predict(vectorizer.transform([name]))[0]
    return {'cate':cate_id_name_dict[pred]}

run(host='0.0.0.0', port=8887)

Bottle v0.12.9 server starting up (using WSGIRefServer())...
Listening on http://0.0.0.0:8887/
Hit Ctrl-C to quit.



classify called


127.0.0.1 - - [24/Aug/2016 06:02:31] "GET /classify?name=[%EC%8B%A0%ED%95%9C%EC%B9%B4%EB%93%9C5%%ED%95%A0%EC%9D%B8][%EC%98%88%ED%99%94-%EC%A2%8B%EC%9D%80%EC%95%84%EC%9D%B4%EB%93%A4]%20%EC%95%84%EB%8F%99%ED%95%9C%EB%B3%B5%20%EC%97%AC%EC%95%84%201076%20%EB%B9%9B%EC%9D%B4%EB%82%98%EB%85%B8%EB%9E%91&img= HTTP/1.1" 200 74


classify called
classify called
classify called
classify called


10.0.2.2 - - [24/Aug/2016 06:03:15] "GET /classify?name=%EC%9D%B8%ED%85%94CPU%EC%BD%94%EC%96%B4%20i3-4%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20%EB%A6%AC%ED%94%84%EB%A0%88%EC%8B%9C%204160(3.6GHz/INTEL)&img=http://shopping.phinf.naver.net/main_7975500/7975500771.jpg HTTP/1.1" 200 62
10.0.2.2 - - [24/Aug/2016 06:03:15] "GET /classify?name=[4%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]%EC%9D%B8%ED%85%94%20intel%20%EC%BD%94%EC%96%B44%EC%84%B8%EB%8C%80%20%ED%95%98%EC%8A%A4%EC%9B%B0%20i3-4160&img=http://shopping.phinf.naver.net/main_8350292/8350292853.jpg HTTP/1.1" 200 62
10.0.2.2 - - [24/Aug/2016 06:03:15] "GET /classify?name=%EC%9D%B8%ED%85%94%20%EC%BD%94%EC%96%B4i3-4%EC%84%B8%EB%8C%80%204160%20(%ED%95%98%EC%8A%A4%EC%9B%B0%20%EB%A6%AC%ED%94%84%EB%A0%88%EC%8B%9C)%20(%EB%B2%8C%ED%81%AC)&img=http://shopping.phinf.naver.net/main_7939938/7939938154.jpg HTTP/1.1" 200 62
10.0.2.2 - - [24/Aug/2016 06:03:15] "GET /classify?name=ICODA%20[%EB%8C%80%EB%A6%AC%EC%A0%90%EC%A0%95%ED%9

...

classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called


10.0.2.2 - - [24/Aug/2016 06:04:21] "GET /classify?name=[6%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]%20ATEN%20CS62S/USB%ED%8F%AC%ED%8A%B8&img=http://shopping.phinf.naver.net/main_8276229/8276229031.jpg HTTP/1.1" 200 102
10.0.2.2 - - [24/Aug/2016 06:04:21] "GET /classify?name=[AK%20PLAZA][%ED%8B%B0%ED%8C%8C%EB%8B%88]%EB%B6%80%EC%97%89%EC%9D%B4%20%EB%B8%94%EB%9E%99%20%EB%82%98%EC%97%BC%20%EB%A0%88%EA%B9%85%EC%8A%A4%20T44KCP030&img=http://shopping.phinf.naver.net/main_8132393/8132393469.jpg HTTP/1.1" 200 80
10.0.2.2 - - [24/Aug/2016 06:04:21] "GET /classify?name=NETmate%20%EC%9D%BC%EC%B2%B4%ED%98%95%20USB%20KVM%2021%20%EC%8A%A4%EC%9C%84%EC%B9%98%20[PCD0994]&img=http://shopping.phinf.naver.net/main_8277532/8277532700.jpg HTTP/1.1" 200 102
10.0.2.2 - - [24/Aug/2016 06:04:21] "GET /classify?name=[%ED%8C%A8%EC%85%98%ED%94%8C%EB%9F%AC%EC%8A%A4][%ED%8B%B0%ED%8C%8C%EB%8B%88%EC%8B%9C%EC%8A%A4%ED%84%B0(AK%20PLAZA)]%ED%8B%B0%ED%8C%8C%EB%8B%88%20%20%EB%B6%80%EC%97%89%EC%9D%B4%20%EB%B8%

classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called


10.0.2.2 - - [24/Aug/2016 06:04:22] "GET /classify?name=[6%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]DC25%20%EC%95%8C%ED%8A%B8%EC%8A%A8%20%EC%9B%90%EA%B2%A9%EC%A0%9C%EC%96%B4%208:1%20KVM%20%EC%8A%A4%EC%9C%84%EC%B2%98%20/KVM%EC%8A%A4%EC%9C%84%EC%B9%98/KV&img=http://shopping.phinf.naver.net/main_8270516/8270516327.jpg HTTP/1.1" 200 102
10.0.2.2 - - [24/Aug/2016 06:04:22] "GET /classify?name=[5%%20%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8](%ED%98%84%EB%8C%80Hmall)(%ED%8C%AC%EC%BD%A7%ED%82%A4%EC%A6%88/%ED%9B%84%EB%93%9C)%20POPMONG%20KIDS%20TYPOGRAPHY%20HOODIE%20%EC%A0%84%EC%BB%AC%EB%9F%AC%20%ED%83%9D1_PKOTIHD54U&img=http://shopping.phinf.naver.net/main_8265242/8265242566.jpg HTTP/1.1" 200 86
10.0.2.2 - - [24/Aug/2016 06:04:22] "GET /classify?name=[4%%EC%A6%89%EC%8B%9C%ED%95%A0%EC%9D%B8%EC%BF%A0%ED%8F%B0]ATEN%202:1%20USB%20%EB%93%80%EC%96%BC%20%EB%B7%B0%20KVM%20%EC%8A%A4%EC%9C%84%EC%B2%98%20/%EB%93%80%EC%96%BC%20KVM&img=http://shopping.phinf.naver.net/main_8283760/8283760359.jpg HTT

classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
classify called
